# Coursework 1: ML basics and fully-connected networks

#### Instructions

Please submit a version of this notebook containing your answers on CATe as *CW1*. Write your answers in the cells below each question.

We recommend that you work on the Ubuntu workstations in the lab. This assignment and all code were only tested to work on these machines. In particular, we cannot guarantee compatibility with Windows machines and cannot promise support if you choose to work on a Windows machine.

You can work from home and use the lab workstations via ssh (for list of machines: https://www.doc.ic.ac.uk/csg/facilities/lab/workstations). 

Once logged in, run the following commands in the terminal to set up a Python environment with all the packages you will need.

    export PYTHONUSERBASE=/vol/bitbucket/nuric/pypi
    export PATH=/vol/bitbucket/nuric/pypi/bin:$PATH

Add the above lines to your `.bashrc` to have these enviroment variables set automatically each time you open your bash terminal.

Any code that you submit will be expected to run in this environment. Marks will be deducted for code that fails to run.

Run `jupyter-notebook` in the coursework directory to launch Jupyter notebook in your default browser.

DO NOT attempt to create a virtualenv in your home folder as you will likely exceed your file quota.

**DEADLINE: 7pm, Tuesday 5th February, 2019**

## Part 1

1. Describe two practical methods used to estimate a supervised learning model's performance on unseen data. Which strategy is most commonly used in most deep learning applications, and why?
2. Suppose that you have reason to believe that your multi-layer fully-connected neural network is overfitting. List four things that you could try to improve generalization performance.

\**ANSWERS FOR PART 1 IN THIS CELL*\*
1. One method is to use split dataset into training, validation and testing set. The model is trained on the training set and the hyperparameters of the model are tuned using the validation set. The performance of the model on unseen data can then be estimated using the testing set. The other method is the cross validation method, where the dataset is split into k-partitions. The model is trained on all of the partitions except one that acts as the testing set. This process is then repeated, creating k-different models. The performance of the model on unseen data can be estimated by taking the average performance of all k models. The training and testing data split is more commonly used in deep learning applications as it is too computational costly to train k different models. Cross validation is only efficient when the dataset is small and if the model is not too complex.
2. Weight Decay - Penalizes large weight parameters using regularization term (L1 or L2 penalty)
   Adding Noise to Input - Adds Gaussian noise to inputs, the variance of the noise will act like a weight decay
   Early Stopping - Stops the learning before reaching convergence (before the performance of model starts to degrade/starts to overfit)
   Dropout - Randomly disable some neurons during training to prevents the network from being too dependent on any one of the neurons

In [ ]:
Adding noise to input -> one type of data augmentation. Otherwise good.

## Part 2

1. Why can gradient-based learning be difficult when using the sigmoid or hyperbolic tangent functions as hidden unit activation functions in deep, fully-connected neural networks?
2. Why is the issue that arises in the previous question less of an issue when using such functions as output unit activation functions, provided that an appropriate loss function is used?
3. What would happen if you initialize all the weights to zero in a multi-layer fully-connected neural network and attempt to train your model using gradient descent? What would happen if you did the same thing for a logistic regression model?

\**ANSWERS FOR PART 2 IN THIS CELL*\*

1. Sigmoid and hyperbolic tangent functions can cause vanishing gradient problem, which happens when the gradient becomes too small (approaching zero), preventing the weights from changing their values.
2. At the output layer, the gradient of the layer is calculated together with the activation function and the loss function. Hence if an appropriate loss function is used, the gradient might not be too small to cause the vanishing gradient problem.
3. If all the weights are initialized to zero, the network will perform poorly as all of the weights will be similar. The weights are updated by gradient descent, which is done through backpropagation. The errors backpropagated are proportional to the value fo the weights. If all the weights are zero, the backpropagated errors will be the same, causing all the weights to be updated to the same value. For logistic regression, there is no issue for initializing all weights to zero as the update rule is simple compared to that of a neural network. As logistic regression is a convex problem, convergence is guaranteed regardless of starting points.

In [ ]:
2.2:
    - Using for e.g. log loss with sigmoid/tanh at output layer will undo the exponentiation in the latter, resulting in a non-negligible gradient being propagated back to earlier layers.

The facts mentioned for 2.3 are correct, but something is missing:
    - Regarding NNs: The fact that all weights are zero, means that all hidden units after first layer will be zero, regardless of the input. All corresponding weights will have gradient zero and as such will not change.
    - Regarding logistic reg.: The gradients of the parameters of the single layer, do, in fact, depend on the input. It is not the starting point that matters.

## Part 3

In this part, you will use PyTorch to implement and train a multinomial logistic regression model to classify MNIST digits.

Restrictions:
* You must use (but not modify) the code provided in `utils.py`. **This file is deliberately not documented**; read it carefully as you will need to understand what it does to complete the tasks.
* You are NOT allowed to use the `torch.nn` module.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultinomialLogisticRegressionClassifier` class below by filling in the missing parts (expected behaviour is prescribed in the documentation):
    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 92% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.


In [2]:
from utils import *
import numpy as np

In [59]:
def log_softmax(x):
    log_softmax = torch.empty(x.shape)
    logsumexp = 0
    for i in range(x.shape[0]):
        b = x[i].max()
        y = np.exp(x[i] - b)
        logsumexp = b + np.log(y.sum())
        log_softmax[i] = x[i] - logsumexp
    return log_softmax

In [81]:
# *CODE FOR PART 3.1 IN THIS CELL*

class MultinomialLogisticRegressionClassifier:
    def __init__(self, weight_init_sd=100.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        self.weight_init_sd = weight_init_sd

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        n = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([self.weight_init_sd]))
        self.W = (n.sample((784,10))).view(784,10)
        self.W.requires_grad_()
        self.b = torch.zeros(10).view(10)
        self.b.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be a Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        inputs = inputs.view(-1, 784)
        h = torch.add(torch.mm(inputs, self.W), self.b)
        # Could not get own log_softmax function to work
        return F.log_softmax(h, dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.W, self.b]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_reg = 0.
        for param in self.W:
            l1_reg += torch.sum(torch.abs(param))
        l1_reg += torch.sum(torch.abs(self.b))
        return l1_reg
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l2_reg = 0.
        for param in self.W:
            l2_reg += torch.sum(torch.pow(param, 2))
        l2_reg += torch.sum(torch.pow(self.b, 2))
        return l2_reg
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [95]:
# *CODE FOR PART 3.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *
model = MultinomialLogisticRegressionClassifier(weight_init_sd=0.01)
res = run_experiment(
    model,
    optimizer=optim.SGD(model.parameters(), 0.05),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.00005,
    l2_penalty_coef=0.00005,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 0.5942, Accuracy: 0.8590
Validation set:	Average loss: 0.4158, Accuracy: 0.8888

Epoch 1: training...
Train set:	Average loss: 0.3841, Accuracy: 0.8960
Validation set:	Average loss: 0.3644, Accuracy: 0.8992

Epoch 2: training...
Train set:	Average loss: 0.3515, Accuracy: 0.9028
Validation set:	Average loss: 0.3429, Accuracy: 0.9060

Epoch 3: training...
Train set:	Average loss: 0.3347, Accuracy: 0.9068
Validation set:	Average loss: 0.3284, Accuracy: 0.9078

Epoch 4: training...
Train set:	Average loss: 0.3240, Accuracy: 0.9104
Validation set:	Average loss: 0.3205, Accuracy: 0.9117

Epoch 5: training...
Train set:	Average loss: 0.3163, Accuracy: 0.9122
Validation set:	Average loss: 0.3150, Accuracy: 0.9147

Epoch 6: training...
Train set:	Average loss: 0.3105, Accuracy: 0.9139
Validation set:	Average loss: 0.3097, Accuracy: 0.9142

Epoch 7: training...
Train set:	Average loss: 0.3060, Accuracy: 0.9153
Validation set:	Average loss: 0.3068, Ac

The hyperparameters used to get a test set accuracy of 92.09% are:

weight_init_sd = 0.01

optimizer = SGD

learning_rate = 0.05

l1_penalty_coef = 0.00005

l2_penalty_coef = 0.00005

In [ ]:
Great code.

## Part 4

In this part, you will use PyTorch to implement and train a multi-layer fully-connected neural network to classify MNIST digits.

Your network must have three hidden layers with 128, 64, and 32 hidden units respectively.

The same restrictions as in Part 3 apply.

Please insert your solutions to the following tasks in the cells below:
1. Complete the `MultilayerClassifier` class below by filling in the missing parts of the following methods (expected behaviour is prescribed in the documentation):

    * The constructor
    * `forward`
    * `parameters`
    * `l1_weight_penalty`
    * `l2_weight_penalty`

2. The default hyperparameters for `MultilayerClassifier` and `run_experiment` have been deliberately chosen to produce poor results. Experiment with different hyperparameters until you are able to get a test set accuracy above 97% after a maximum of 10 epochs of training. However, DO NOT use the test set accuracy to tune your hyperparameters; use the validation loss / accuracy. You can use any optimizer in `torch.optim`.

3. Describe an alternative strategy for initializing weights that may perform better than the strategy we have used here.

In [4]:
# *CODE FOR PART 4.1 IN THIS CELL*

class MultilayerClassifier:
    def __init__(self, activation_fun="sigmoid", weight_init_sd=1.0):
        """
        Initializes model parameters to values drawn from the Normal
        distribution with mean 0 and standard deviation `weight_init_sd`.
        """
        super().__init__()
        self.activation_fun = activation_fun
        self.weight_init_sd = weight_init_sd

        if self.activation_fun == "relu":
            self.activation = F.relu
        elif self.activation_fun == "sigmoid":
            self.activation = torch.sigmoid
        elif self.activation_fun == "tanh":
            self.activation = torch.tanh
        else:
            raise NotImplementedError()

        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        n = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([self.weight_init_sd]))
        self.W1 = n.sample((784,64)).view(784,64)
        self.W1.requires_grad_()
        self.b1 = torch.zeros(64).view(64)
        self.b1.requires_grad_()
        self.W2 = n.sample((64,32)).view(64,32)
        self.W2.requires_grad_()
        self.b2 = torch.zeros(32).view(32)
        self.b2.requires_grad_()
        self.W3 = n.sample((32,10)).view(32,10)
        self.W3.requires_grad_()
        self.b3 = torch.zeros(10).view(10)
        self.b3.requires_grad_()
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def forward(self, inputs):
        """
        Performs the forward pass through the model.
        
        Expects `inputs` to be Tensor of shape (batch_size, 1, 28, 28) containing
        minibatch of MNIST images.
        
        Inputs should be flattened into a Tensor of shape (batch_size, 784),
        before being fed into the model.
        
        Should return a Tensor of logits of shape (batch_size, 10).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        inputs = inputs.view(-1, 784)
        h = torch.add(torch.mm(inputs, self.W1), self.b1)
        h = torch.add(torch.mm(h, self.W2), self.b2)
        h = torch.add(torch.mm(h, self.W3), self.b3)
        return F.log_softmax(h, dim=1)
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def parameters(self):
        """
        Should return an iterable of all the model parameter Tensors.
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        return [self.W1, self.b1, self.W2, self.b2, self.W3, self.b3]
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################
        
    
    def l1_weight_penalty(self):
        """
        Computes and returns the L1 norm of the model's weight vector (i.e. sum
        of absolute values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l1_reg = 0.
        for param in self.W1:
            l1_reg += torch.sum(torch.abs(param))
        for param in self.W2:
            l1_reg += torch.sum(torch.abs(param))
        for param in self.W3:
            l1_reg += torch.sum(torch.abs(param))
        l1_reg += torch.sum(torch.abs(self.b1))
        l1_reg += torch.sum(torch.abs(self.b2))
        l1_reg += torch.sum(torch.abs(self.b3))
        return l1_reg
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################

    def l2_weight_penalty(self):
        """
        Computes and returns the L2 weight penalty (i.e. 
        sum of squared values of all model parameters).
        """
        #######################################################################
        #                       ** START OF YOUR CODE **
        #######################################################################
        l2_reg = 0.
        for param in self.W1:
            l2_reg += torch.sum(torch.pow(param, 2))
        for param in self.W2:
            l2_reg += torch.sum(torch.pow(param, 2))
        for param in self.W3:
            l2_reg += torch.sum(torch.pow(param, 2))
        l2_reg += torch.sum(torch.pow(self.b1, 2))
        l2_reg += torch.sum(torch.pow(self.b2, 2))
        l2_reg += torch.sum(torch.pow(self.b3, 2))
        return l2_reg
        #######################################################################
        #                       ** END OF YOUR CODE **
        #######################################################################


In [12]:
# *CODE FOR PART 4.2 IN THIS CELL - EXAMPLE WITH DEFAULT PARAMETERS PROVIDED *

model = MultilayerClassifier(activation_fun='sigmoid', weight_init_sd=0.02)
res = run_experiment(
    model,
    optimizer=optim.SGD(model.parameters(), 0.05),
    train_loader=train_loader_0,
    val_loader=val_loader_0,
    test_loader=test_loader_0,
    n_epochs=10,
    l1_penalty_coef=0.00001,
    l2_penalty_coef=0.00001,
    suppress_output=False
)

Epoch 0: training...
Train set:	Average loss: 1.3339, Accuracy: 0.5462
Validation set:	Average loss: 0.4857, Accuracy: 0.8603

Epoch 1: training...
Train set:	Average loss: 0.4217, Accuracy: 0.8783
Validation set:	Average loss: 0.3706, Accuracy: 0.8933

Epoch 2: training...
Train set:	Average loss: 0.3595, Accuracy: 0.8994
Validation set:	Average loss: 0.3402, Accuracy: 0.9050

Epoch 3: training...
Train set:	Average loss: 0.3329, Accuracy: 0.9069
Validation set:	Average loss: 0.3274, Accuracy: 0.9087

Epoch 4: training...
Train set:	Average loss: 0.3153, Accuracy: 0.9109
Validation set:	Average loss: 0.3050, Accuracy: 0.9178

Epoch 5: training...
Train set:	Average loss: 0.3043, Accuracy: 0.9144
Validation set:	Average loss: 0.2995, Accuracy: 0.9145

Epoch 6: training...
Train set:	Average loss: 0.2952, Accuracy: 0.9167
Validation set:	Average loss: 0.2911, Accuracy: 0.9207

Epoch 7: training...
Train set:	Average loss: 0.2895, Accuracy: 0.9195
Validation set:	Average loss: 0.2887, Ac

The hyperparameters used to get a test set accuracy of 92.11% are:

weight_init_sd = 0.02

optimizer = SGD

learning_rate = 0.05

l1_penalty_coef = 0.00001

l2_penalty_coef = 0.00001

\**ANSWERS FOR PART 4.3 IN THIS CELL*\*

An alternative strategy for initializing weights is to initialize the weights randomly, but differing in range depending on the size of the previous layer of neurons.

In [ ]:
Great code.

As for 4.3: Perhaps you refer at the He et al. 2015 initialisation?